In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks

In [7]:
!pip install openpyxl


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
df = pd.read_excel("../Data/Raw/cattle_analysis.xlsx")

In [9]:
X = df.drop(columns=["AnimalNo", "ActualBodyWeight"])
y = df["ActualBodyWeight"]

In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [11]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [12]:
model = models.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)  # single output for regression
])

model.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae']
)

c:\Users\gauta\OneDrive\Dokumentai\cattle_model\.venv\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [13]:
early_stop = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=200,
    batch_size=16,
    callbacks=[early_stop],
    verbose=1
)

Epoch 1/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 366946.0938 - mae: 601.1447 - val_loss: 380729.5312 - val_mae: 611.7285
Epoch 2/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 365701.8125 - mae: 600.0903 - val_loss: 379284.6250 - val_mae: 610.5366
Epoch 3/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 363962.2188 - mae: 598.6050 - val_loss: 376995.0000 - val_mae: 608.6497
Epoch 4/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 361113.8750 - mae: 596.1593 - val_loss: 373173.0625 - val_mae: 605.5002
Epoch 5/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 356478.3438 - mae: 592.1885 - val_loss: 367065.5312 - val_mae: 600.4250
Epoch 6/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 349225.0625 - mae: 585.8967 - val_loss: 357623.5312 - val_mae: 592.5383
Epoch 7/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 338524.4062 - mae: 576.4720 - val_loss: 343944.0625 - val_mae: 580.9738
Epoch 8/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 323627.4375 - mae: 562.9

In [14]:
loss, mae = model.evaluate(X_test, y_test, verbose=0)
print(f"Test MAE: {mae:.2f}")
print(f"Test MSE: {loss:.2f}")

Test MAE: 27.58
Test MSE: 1192.56


In [15]:
predictions = model.predict(X_test)
pred_df = pd.DataFrame({
    "Actual": y_test.values,
    "Predicted": predictions.flatten()
})

print(pred_df.head())

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
   Actual   Predicted
0     617  658.678040
1     756  763.529663
2     682  617.341003
3     559  594.665100
4     757  737.055847


In [17]:
pred_df.to_csv("../Results/nn_predictions.csv", index=False)

In [18]:
model.save("../Models/cattle_nn_model.h5")
print("Model saved to Models/cattle_nn_model.h5")

Model saved to Models/cattle_nn_model.h5


In [19]:
from sklearn.metrics import r2_score

In [21]:
r2 = r2_score(y_test, predictions)
print(f"R-squared: {r2:.4f}")

R-squared: 0.8383
